# Tesserae v5 Demo

This demo will go over the basics of Tesserae v5 development up through July 19, 2018.

In [ ]:
import json

from tesserae.db import get_connection, Text, Unit, Token
from tesserae.text_access.storage import retrieve_text_list, insert_text, load_text
from tesserae.tokenizers.tokenize import get_token_info
from tesserae.tokenizers.storage import retrieve_token_list, insert_tokens
from tesserae.unitizers import poetry, prose
from tesserae.unitizers.storage import retrieve_unit_list, insert_units

client = get_connection('45.55.219.221', 27017, None, None, 'tesstest')
client['texts'].delete_many({}) # Clean up from previous demo

## Inserting Texts

Inserting texts requires the user to provide text metadata, including the CTS URN, title, author, and filepath. We'll start by loading text metadata.

In [ ]:
with open('text_metadata.json', 'r') as f:
    text_meta = json.load(f)

print('{}{}{}{}'.format('Title'.ljust(15), 'Author'.ljust(15), 'Language'.ljust(15), 'Year'))
for t in text_meta:
    print('{}{}{}{}'.format(t['title'].ljust(15), t['author'].ljust(15), t['language'].ljust(15), str(t['year']).ljust(15)))

Then insert the new texts with `tesserae.text_access.insert_text`

In [ ]:
client.texts.delete_many({})
for t in text_meta:
    result = insert_text(client,
                         cts_urn=t['cts_urn'],
                         author=t['author'],
                         title=t['title'],
                         language=t['language'],
                         year=t['year'],
                         unit_types=t['unit_types'],
                         path=t['path'])
    print(result.inserted_id)

We can retrieve the inserted texts with `tesserae.text_access.retrieve_text_list`. These texts will be converted to objects representing the database entries. The returned text list can be filtered by any valid field in the text database.

In [ ]:
texts = retrieve_text_list(client)

print('{}{}{}{}'.format('Title'.ljust(15), 'Author'.ljust(15), 'Language'.ljust(15), 'Year'))
for t in texts:
    print('{}{}{}{}'.format(t.title.ljust(15), t.author.ljust(15), t.language.ljust(15), t.year))

## Loading .tess Files

Text metadata includes the path to the .tess file on the local filesystem. Using a Text retrieved from the database, the file can be loaded for further processing.

In [ ]:
tessfile = load_text(client, texts[1].cts_urn)

print(tessfile.path)
print(len(tessfile))
print(tessfile[270])

We can iterate through the file line-by-line.

In [ ]:
lines = tessfile.readlines()
for i in range(10):
    print(next(lines))

We can also iterate token-by-token.

In [ ]:
tokens = tessfile.read_tokens()
for i in range(10):
    print(next(tokens))

## Tokenizing a Text

Texts can be tokenized with `tesserae.tokenizers.get_token_info`. This function takes a token and the language to use for lemmatization, etc.

In [ ]:
tokenized = []
print('{}{}{}'.format('Raw'.ljust(15), 'Normalized'.ljust(15), 'Lemmata'))
for i in range(10):
    token = get_token_info(next(tokens), tessfile.metadata.language)
    print('{}{}{}'.format(token.raw.ljust(15), token.token_type.ljust(15), token.lemmata))
    tokenized.append(token)

Processed tokens can then be stored in and retrieved from the database, similar to text metadata. It should be noted that the resulting list is shorter than the original token list. During insertion, Tesserae removes duplicate tokens to prevent database bloat.

In [ ]:
client['tokens'].delete_many({})
result = insert_tokens(client, tokenized)

tokens = retrieve_token_list(client)
for token in tokens:
    print('{}{}{}'.format(token.raw.ljust(15), token.token_type.ljust(15), token.lemmata))

## Unitizing a Text

Texts can be unitized into lines (poetry only) and phrases (poetry and prose), and the intertext matches are found between units of text.


In [ ]:
# Unitizing lines of a poem
if tessfile.metadata.author in ['vergil', 'euripides']:
    units = poetry.split_line_units(tessfile)
    for i in range(10):
        print(units[i].raw)

In [ ]:
# Unitizing phrases of a poem or prose
if tessfile.metadata.author in ['vergil', 'euripides']:
    units = poetry.split_phrase_units(tessfile)
    for i in range(10):
        print(units[i].raw)
else:
    units = prose.split_phrase_units(tessfile)
    for i in range(10):
        print(units[i].raw)